In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd


import sys
sys.path.append('./waveglow/')
sys.path.append('./waveglow/tacotron2')

from denoiser import Denoiser
import librosa
import numpy as np
import os
import glob
import json

import torch
from text import text_to_sequence, cmudict
from text.symbols import symbols
import commons
import attentions
import modules
import models
import utils


# load waveglow
#waveglow_path = './waveglow/waveglow_256channels_universal_v5.pt'
#waveglow_path = './waveglow/waveglow_9000'
#waveglow_path = '/home/sean/Downloads/waveglow_12000'
waveglow_path = '/home/sean/Downloads/waveglow_23600'
#waveglow_path = '/home/sean/src/waveglow/checkpoints/waveglow_42000'
#waveglow_path = '/home/sean/Downloads/waveglow_90400'
#waveglow_path = '/home/sean/Downloads/waveglow_27400'
#waveglow_path = '/home/sean/Downloads/waveglow_40800'

waveglow = torch.load(waveglow_path, map_location='cpu')['model']
#waveglow = waveglow.remove_weightnorm(waveglow)
_ = waveglow.eval()
#from apex import amp
#waveglow, _ = amp.initialize(waveglow, [], opt_level="O3") # Try if you want to boost up synthesis speed.
#denoiser = Denoiser(waveglow)

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [6]:
model_dir = "./logs/obama/"
hps = utils.get_hparams_from_dir(model_dir)
model = models.FlowGenerator(
    len(symbols),
    out_channels=hps.data.n_mel_channels,
    **hps.model).to("cpu")

checkpoint_path = utils.latest_checkpoint_path(model_dir)
#checkpoint_path = "./logs/trump/G_360.pth"
utils.load_checkpoint(checkpoint_path, model)
model.decoder.store_inverse() # do not calcuate jacobians for fast decoding
_ = model.eval()

cmu_dict = cmudict.CMUDict(hps.data.cmudict_path)

./logs/obama/G_1845.pth
INFO:root:Loaded checkpoint './logs/obama/G_1845.pth' (iteration 1845)


In [18]:
next(model.parameters()).is_cuda

False

In [16]:
tst_stn = "We're rounding 'em up in a very humane way, in a very nice way. And they're going to be happy because they want to be legalized. And, by the way, I know it doesn't sound nice. But not everything is nice." # Adding spaces at the beginning and the end of utterance improves quality
#tst_stn = "What I won't do is take in two hundred thousand Syrians who could be ISIS... I have been watching this migration. And I see the people. I mean, they're men. They're mostly men, and they're strong men. These are physically young, strong men. They look like prime-time soldiers. Now it's probably not true, but where are the women?... So, you ask two things. Number one, why aren't they fighting for their country? And number two, I don't want these people coming over here."
#tst_stn = "Are you a great president? Like, from the standpoint of the other side, I think I win in a landslide because of what we did. I think we’re going to have a very interesting election. I think it’s going to be very interesting. Whoever it is, I’m going to be the best candidate that they can have. I’m not going to change. I’m going to be the best candidate that they can have, because there’s nobody that loves the country as much or more than me."
#tst_stn = " Longer pauses between sentences.How about this?"
#tst_stn = " I have small hands. Really, they are so small. "
#tst_stn = " Okay, so I'm testing a new model.  What do you think? "
#tst_stn = " This new model can produce much longer sentences, and it seems to handle more difficult words better. "
#tst_stn = " Do you think people would pay money to talk to me? "
#tst_stn = " Hello, this is Donald Trump. "
tst_stn = " Am I a great President? ..... Hell yes I am. Now what can I help you with? "
#tst_stn = " Is this thing on?  Eric! Get out of here! .... This is the president. "
tst_stn = " How much money are you paying me to talk to you? "
#tst_stn = " First off, I don't care if you don't like me. So what do you want? ..."
tst_stn = " Change will not come if we wait for some other person or some other time. We are the ones we've been waiting for. We are the change that we seek. "
sequence = np.array(text_to_sequence(tst_stn, ['english_cleaners'], cmu_dict))[None, :]
print("".join([symbols[c] for c in sequence[0]]))
x_tst = torch.autograd.Variable(torch.from_numpy(sequence)).to('cpu').long()
x_tst_lengths = torch.tensor([x_tst.shape[1]])

 @CH@EY1@N@JH @W@IH1@L @N@AA1@T @K@AH1@M @IH1@F @W@IY1 @W@EY1@T @F@AO1@R @S@AH1@M @AH1@DH@ER0 @P@ER1@S@AH0@N @AO1@R @S@AH1@M @AH1@DH@ER0 time. @W@IY1 @AA1@R @DH@AH0 @W@AH1@N@Z @W@IY1@V @B@IH1@N @W@EY1@T@IH0@NG for. @W@IY1 @AA1@R @DH@AH0 @CH@EY1@N@JH @DH@AE1@T @W@IY1 seek. 


In [19]:
with torch.no_grad():
  #noise_scale = 0.000015
  #noise_scale = 0.9
  noise_scale = 0.667
  length_scale = 1.1
  (y_gen_tst, *r), attn_gen, *_ = model(x_tst, x_tst_lengths, gen=True, noise_scale=noise_scale, length_scale=length_scale)
  #try:
  #  audio = waveglow.infer(y_gen_tst.half(), sigma=0.666)#.666)
  #except:
  audio = waveglow.infer(y_gen_tst, sigma=.666)
#ipd.Audio(audio.data.cpu().numpy(), rate=hps.data.sampling_rate)


RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable

In [17]:
audio_denoised = denoiser(audio, strength=0.01)
ipd.Audio(audio_denoised.data.cpu().numpy(), rate=hps.data.sampling_rate)